# Optimal Routing Problem

This Notebook will solve a vehicle routing problem for two routes in Minneapolis. 

## Setup and getting data. 

In [1]:
import arcpy
import requests


In [2]:
arcpy.env.workspace = "D:/users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/Lab3_Networking.gdb"
arcpy.env.overwriteOutput = True

In [3]:
base_url = base_url = "https://gisdata.mn.gov/api/3/action/package_search?q="

tag = "roads"

In [4]:

package_information = requests.get(base_url + tag, auth=('user', 'pass'), verify= False) ##Requsting the result from the search inquiry. 

D:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gisdata.mn.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [5]:
r = requests.get("https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/trans_road_centerlines_gac/shp_trans_road_centerlines_gac.zip")

open('shp_trans_road_centerlines_gac.zip', 'wb').write(r.content)
import zipfile
with zipfile.ZipFile('shp_trans_road_centerlines_gac.zip', 'r') as zip_ref:
    zip_ref.extractall("d:/git/GIS5572shpfiles/ShapefilesLab3/Roadcenterlines.shp")

In [24]:
shp_path = "d:/git/GIS5572shpfiles/ShapefilesLab3/RoadCenterline.shp"  ## This  is the shapefile


### Barrier 

This next cell will create a barrier with 35W and 94 but still allow streets through.

In [7]:
d = arcpy.management.SelectLayerByAttribute(shp_path, "NEW_SELECTION", "ROUTE_NUM = '94' Or ROUTE_NUM = '35W'", None)

arcpy.management.CopyFeatures(d, "Protobarrier") ## Used for the Barrier


d = arcpy.management.SelectLayerByAttribute(shp_path, "NEW_SELECTION", "ROUTE_NUM = '94' Or ROUTE_NUM = '35W'", "INVERT")

arcpy.management.CopyFeatures(d, "gaps") ## Used to erase the barrier with a 1 meter buffer

arcpy.analysis.Buffer("gaps", r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\Lab3_Networking.gdb\gaps_Buff", "1 Meters", "FULL", "ROUND", "NONE", None, "PLANAR")
arcpy.Erase_analysis("Protobarrier", "gaps_Buff", "Barrier")


<Result 'D:/Users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/Lab3_Networking.gdb\\Barrier'>

## Setting up the Network Dataset

In [17]:
arcpy.management.CreateFeatureDataset(r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\Lab3_Networking.gdb", "Transportation", "PROJCS['NAD_1983_UTM_Zone_15N',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-93.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]];-5120900 -9998100 10000;-100000 10000;-100000 10000;0.001;0.001;0.001;IsHighPrecision")

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\Lab3_Networking.gdb\\Transportation'>

In [18]:
arcpy.conversion.FeatureClassToFeatureClass(shp_path, r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\Lab3_Networking.gdb\Transportation", "Streets", '', 'ROADSEG_ID "ROADSEG_ID" true true false 36 Text 0 0,First,#,RoadCenterline,ROADSEG_ID,0,36;ROUTE_ID "ROUTE_ID" true true false 18 Text 0 0,First,#,RoadCenterline,ROUTE_ID,0,18;ROUTE_SYS "ROUTE_SYS" true true false 2 Text 0 0,First,#,RoadCenterline,ROUTE_SYS,0,2;ROUTE_DIR "ROUTE_DIR" true true false 1 Text 0 0,First,#,RoadCenterline,ROUTE_DIR,0,1;LOC_STATE "LOC_STATE" true true false 10 Text 0 0,First,#,RoadCenterline,LOC_STATE,0,10;PRIME_STAT "PRIME_STAT" true true false 10 Text 0 0,First,#,RoadCenterline,PRIME_STAT,0,10;ST_PRE_MOD "ST_PRE_MOD" true true false 15 Text 0 0,First,#,RoadCenterline,ST_PRE_MOD,0,15;ST_PRE_DIR "ST_PRE_DIR" true true false 9 Text 0 0,First,#,RoadCenterline,ST_PRE_DIR,0,9;ST_PRE_TYP "ST_PRE_TYP" true true false 35 Text 0 0,First,#,RoadCenterline,ST_PRE_TYP,0,35;ST_PRE_SEP "ST_PRE_SEP" true true false 20 Text 0 0,First,#,RoadCenterline,ST_PRE_SEP,0,20;ST_NAME "ST_NAME" true true false 60 Text 0 0,First,#,RoadCenterline,ST_NAME,0,60;ST_POS_TYP "ST_POS_TYP" true true false 15 Text 0 0,First,#,RoadCenterline,ST_POS_TYP,0,15;ST_POS_DIR "ST_POS_DIR" true true false 9 Text 0 0,First,#,RoadCenterline,ST_POS_DIR,0,9;ST_POS_MOD "ST_POS_MOD" true true false 15 Text 0 0,First,#,RoadCenterline,ST_POS_MOD,0,15;ST_CONCAT "ST_CONCAT" true true false 150 Text 0 0,First,#,RoadCenterline,ST_CONCAT,0,150;ST_NAME_A1 "ST_NAME_A1" true true false 150 Text 0 0,First,#,RoadCenterline,ST_NAME_A1,0,150;A1_MSAG_V "A1_MSAG_V" true true false 7 Text 0 0,First,#,RoadCenterline,A1_MSAG_V,0,7;ST_NAME_A2 "ST_NAME_A2" true true false 150 Text 0 0,First,#,RoadCenterline,ST_NAME_A2,0,150;A2_MSAG_V "A2_MSAG_V" true true false 7 Text 0 0,First,#,RoadCenterline,A2_MSAG_V,0,7;ST_NAME_A3 "ST_NAME_A3" true true false 150 Text 0 0,First,#,RoadCenterline,ST_NAME_A3,0,150;A3_MSAG_V "A3_MSAG_V" true true false 7 Text 0 0,First,#,RoadCenterline,A3_MSAG_V,0,7;ADD_FR_L "ADD_FR_L" true true false 10 Long 0 10,First,#,RoadCenterline,ADD_FR_L,-1,-1;ADD_TO_L "ADD_TO_L" true true false 10 Long 0 10,First,#,RoadCenterline,ADD_TO_L,-1,-1;ADD_FR_R "ADD_FR_R" true true false 10 Long 0 10,First,#,RoadCenterline,ADD_FR_R,-1,-1;ADD_TO_R "ADD_TO_R" true true false 10 Long 0 10,First,#,RoadCenterline,ADD_TO_R,-1,-1;PARITY_L "PARITY_L" true true false 4 Text 0 0,First,#,RoadCenterline,PARITY_L,0,4;PARITY_R "PARITY_R" true true false 4 Text 0 0,First,#,RoadCenterline,PARITY_R,0,4;ZIP_L "ZIP_L" true true false 5 Text 0 0,First,#,RoadCenterline,ZIP_L,0,5;ZIP_R "ZIP_R" true true false 5 Text 0 0,First,#,RoadCenterline,ZIP_R,0,5;CTU_NAME_L "CTU_NAME_L" true true false 100 Text 0 0,First,#,RoadCenterline,CTU_NAME_L,0,100;CTU_NAME_R "CTU_NAME_R" true true false 100 Text 0 0,First,#,RoadCenterline,CTU_NAME_R,0,100;CTU_ID_L "CTU_ID_L" true true false 8 Text 0 0,First,#,RoadCenterline,CTU_ID_L,0,8;CTU_ID_R "CTU_ID_R" true true false 8 Text 0 0,First,#,RoadCenterline,CTU_ID_R,0,8;POSTCOMM_L "POSTCOMM_L" true true false 40 Text 0 0,First,#,RoadCenterline,POSTCOMM_L,0,40;POSTCOMM_R "POSTCOMM_R" true true false 40 Text 0 0,First,#,RoadCenterline,POSTCOMM_R,0,40;CO_CODE_L "CO_CODE_L" true true false 5 Text 0 0,First,#,RoadCenterline,CO_CODE_L,0,5;CO_CODE_R "CO_CODE_R" true true false 5 Text 0 0,First,#,RoadCenterline,CO_CODE_R,0,5;CO_NAME_L "CO_NAME_L" true true false 40 Text 0 0,First,#,RoadCenterline,CO_NAME_L,0,40;CO_NAME_R "CO_NAME_R" true true false 40 Text 0 0,First,#,RoadCenterline,CO_NAME_R,0,40;STATE_L "STATE_L" true true false 2 Text 0 0,First,#,RoadCenterline,STATE_L,0,2;STATE_R "STATE_R" true true false 2 Text 0 0,First,#,RoadCenterline,STATE_R,0,2;ELEV_FROM "ELEV_FROM" true true false 5 Long 0 5,First,#,RoadCenterline,ELEV_FROM,-1,-1;ELEV_TO "ELEV_TO" true true false 5 Long 0 5,First,#,RoadCenterline,ELEV_TO,-1,-1;ONEWAY "ONEWAY" true true false 1 Text 0 0,First,#,RoadCenterline,ONEWAY,0,1;SPEED_IMP "SPEED_IMP" true true false 5 Long 0 5,First,#,RoadCenterline,SPEED_IMP,-1,-1;EMERG_ACC "EMERG_ACC" true true false 10 Text 0 0,First,#,RoadCenterline,EMERG_ACC,0,10;ROUTESPEED "ROUTESPEED" true true false 5 Long 0 5,First,#,RoadCenterline,ROUTESPEED,-1,-1;ROUTE_NUM "ROUTE_NUM" true true false 5 Text 0 0,First,#,RoadCenterline,ROUTE_NUM,0,5;GIS911POC "GIS911POC" true true false 75 Text 0 0,First,#,RoadCenterline,GIS911POC,0,75;ESN_L "ESN_L" true true false 5 Text 0 0,First,#,RoadCenterline,ESN_L,0,5;ESN_R "ESN_R" true true false 5 Text 0 0,First,#,RoadCenterline,ESN_R,0,5;PSAP_L "PSAP_L" true true false 5 Text 0 0,First,#,RoadCenterline,PSAP_L,0,5;PSAP_R "PSAP_R" true true false 5 Text 0 0,First,#,RoadCenterline,PSAP_R,0,5;MSAG_C_L "MSAG_C_L" true true false 30 Text 0 0,First,#,RoadCenterline,MSAG_C_L,0,30;MSAG_C_R "MSAG_C_R" true true false 30 Text 0 0,First,#,RoadCenterline,MSAG_C_R,0,30;VALID_L "VALID_L" true true false 10 Text 0 0,First,#,RoadCenterline,VALID_L,0,10;VALID_R "VALID_R" true true false 10 Text 0 0,First,#,RoadCenterline,VALID_R,0,10;VERROR_911 "VERROR_911" true true false 10 Text 0 0,First,#,RoadCenterline,VERROR_911,0,10;STATUS "STATUS" true true false 20 Text 0 0,First,#,RoadCenterline,STATUS,0,20;EFF_DATE "EFF_DATE" true true false 8 Date 0 0,First,#,RoadCenterline,EFF_DATE,-1,-1;RET_DATE "RET_DATE" true true false 8 Date 0 0,First,#,RoadCenterline,RET_DATE,-1,-1;EDIT_ORG "EDIT_ORG" true true false 40 Text 0 0,First,#,RoadCenterline,EDIT_ORG,0,40;EDIT_DATE "EDIT_DATE" true true false 8 Date 0 0,First,#,RoadCenterline,EDIT_DATE,-1,-1;FUNCLS_FED "FUNCLS_FED" true true false 1 Text 0 0,First,#,RoadCenterline,FUNCLS_FED,0,1;FUNCLS_MET "FUNCLS_MET" true true false 3 Text 0 0,First,#,RoadCenterline,FUNCLS_MET,0,3;SURF_TYPE "SURF_TYPE" true true false 32 Text 0 0,First,#,RoadCenterline,SURF_TYPE,0,32;NUM_LANES "NUM_LANES" true true false 5 Long 0 5,First,#,RoadCenterline,NUM_LANES,-1,-1;COMMENTS "COMMENTS" true true false 254 Text 0 0,First,#,RoadCenterline,COMMENTS,0,254;SHAPE_Leng "SHAPE_Leng" true true false 19 Double 0 0,First,#,RoadCenterline,SHAPE_Leng,-1,-1', '') 

#Outputing the centerline shapefile to a feature class called "streets"

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\Lab3_Networking.gdb\\Transportation\\Streets'>

In [19]:
arcpy.na.CreateNetworkDataset("Transportation", "min_net","Streets") #Dataset, network name, feature name


<Result 'D:/Users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/Lab3_Networking.gdb\\Transportation\\min_net'>

In [25]:
desc = arcpy.Describe("Transportation/min_net")

print("Network type:   " + desc.networkType)
print("Supports turns? " + str(desc.supportsTurns))
print("Supports directions? " + str(desc.supportsDirections))
print("Is buildable?   " + str(desc.isBuildable))
print("Elevation model: " + desc.elevationModel)
print("Supports historical traffic data: " + str(desc.supportsHistoricalTrafficData))
print("Time zone attribute name: " + desc.timeZoneAttributeName)
print("Time zone table name: " + desc.timeZoneTableName)
print("Optimizations: " + ", ".join(desc.optimizations))

Network type:   Geodatabase
Supports turns? True
Supports directions? True
Is buildable?   True
Elevation model: Elevation Fields
Supports historical traffic data: False
Time zone attribute name: 
Time zone table name: 
Optimizations: 


In [26]:
arcpy.na.BuildNetwork("Transportation/min_net") ## Properies done in Arc Pro

<Result 'D:/Users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/Lab3_Networking.gdb\\Transportation/min_net'>

## Setting up Vehicle Routing Problem 

In [44]:
arcpy.na.MakeVehicleRoutingProblemAnalysisLayer("Transportation/min_net", "Vehicle Routing Problem", "New Travel Mode", "Minutes", "Meters", "12/23/2019", "LOCAL_TIME_AT_LOCATIONS", "ALONG_NETWORK", "Medium", "Medium", "DIRECTIONS", "CLUSTER")

<Result 'Vehicle Routing Problem'>

In [45]:
arcpy.conversion.ExcelToTable("D:/git/GIS5572/Lab3/csv/orders.xlsx") ## Orders address

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\Lab3_Networking.gdb\\orders_ExcelToTable3'>

In [47]:
arcpy.geocoding.GeocodeAddresses(r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\Lab3_Networking.gdb\orders_ExcelToTable3", "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/ArcGIS World Geocoding Service", "'Single Line Input' Address VISIBLE NONE", r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\orders_Geocoded.shp", "STATIC", "US", "ROUTING_LOCATION", None, "ALL")

##Geocode orders

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\orders_Geocoded.shp'>

In [50]:
arcpy.conversion.ExcelToTable("D:/git/GIS5572/Lab3/csv/depot.xlsx") ##Depot address

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\Lab3_Networking.gdb\\depot_ExcelToTable2'>

In [51]:
arcpy.geocoding.GeocodeAddresses(r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\Lab3_Networking.gdb\depot_ExcelToTable", "https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/ArcGIS World Geocoding Service", "'Single Line Input' Address VISIBLE NONE", r"D:\Users\Owner\Documents\ArcGIS\Projects\Lab3_Networking\depot_Geocoded.shp", "STATIC", "US", "ROUTING_LOCATION", None, "ALL")

##Geocoding Depot

<Result 'D:\\Users\\Owner\\Documents\\ArcGIS\\Projects\\Lab3_Networking\\depot_Geocoded.shp'>

In [52]:


arcpy.management.Dissolve("Barrier", "Barrier_D", "ROUTE_ID") ##reduced the number of ids.

<Result 'D:/Users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/Lab3_Networking.gdb\\Barrier_D'>

In [53]:
depot = 'D:/Users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/depot_Geocoded.shp' ## Adding the depot to the vehicle routing problem
arcpy.na.AddLocations("Vehicle Routing Problem", "Depots", depot, "Name USER_Name #;Description # #;TimeWindowStart # #;TimeWindowEnd # #;TimeWindowStart2 # #;TimeWindowEnd2 # #;CurbApproach # 0", "5000 Meters", None, "Streets SHAPE;min_net_Junctions NONE", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "5 Meters", "EXCLUDE", None)

<Result 'Vehicle Routing Problem'>

In [54]:
orders_geocoded = "D:/users/Owner/Documents/ArcGIS/Projects/Lab3_Networking/orders_Geocoded.shp" ## Added the orders (had to manually put in the 10am to 11am exceptions.
arcpy.na.AddLocations("Vehicle Routing Problem", "Orders", orders_geocoded, "Name IN_SingleL #;Description # #;ServiceTime # #;TimeWindowStart # '12/23/2019 8:00 AM';TimeWindowEnd # #;MaxViolationTime # #;TimeWindowStart2 # #;TimeWindowEnd2 # #;MaxViolationTime2 # #;InboundArriveTime # #;OutboundDepartTime # #;DeliveryQuantity_1 # #;DeliveryQuantity_2 # #;DeliveryQuantity_3 # #;DeliveryQuantity_4 # #;DeliveryQuantity_5 # #;DeliveryQuantity_6 # #;DeliveryQuantity_7 # #;DeliveryQuantity_8 # #;DeliveryQuantity_9 # #;PickupQuantity_1 # #;PickupQuantity_2 # #;PickupQuantity_3 # #;PickupQuantity_4 # #;PickupQuantity_5 # #;PickupQuantity_6 # #;PickupQuantity_7 # #;PickupQuantity_8 # #;PickupQuantity_9 # #;Revenue # #;AssignmentRule # 3;RouteName # #;Sequence # #;CurbApproach # 0", "5000 Meters", None, "Streets SHAPE;min_net_Junctions NONE", "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "5 Meters", "EXCLUDE", None)

<Result 'Vehicle Routing Problem'>

In [55]:
arcpy.na.AddLocations("Vehicle Routing Problem", "Line Barriers", "Barrier_D", "Name # #;BarrierType ROUTE_NUM 0;Attr_Minutes # 1;Attr_TravelTime # 1;Attr_Miles # 1;Attr_Kilometers # 1;Attr_TimeAt1KPH # 1;Attr_WalkTime # 1;Attr_TruckMinutes # 1;Attr_TruckTravelTime # 1;Shape_Length Shape_Length #", "5000 Meters", None, None, "MATCH_TO_CLOSEST", "APPEND", "NO_SNAP", "5 Meters", "EXCLUDE", None)

##Adding the Line Barriers 

<Result 'Vehicle Routing Problem'>

In [56]:
arcpy.na.AddVehicleRoutingProblemRoutes("Vehicle Routing Problem ", 2, "Route", "Warehouse", "Warehouse", "12/23/2019 8:00:00 AM", "12/23/2019 8:00:00 AM", 5, None, None, None, None, "APPEND")

##Adding the routes. For equity, I set the max orders to 5 

<Result 'Vehicle Routing Problem'>

In [57]:
arcpy.na.SolveVehicleRoutingProblem('orders', 'depots', 'routes', "",'Minutes', 'Meters', "Transportation/min_net", '', '', '', '', '', "12/23/2019", line_barriers = "Barrier")

## Solving the problem. Output is two routes

id,value
0,true
1,in_memory\UnassignedStops
2,in_memory\Stops
3,in_memory\Routes
4,in_memory\Directions
5,
6,


In [58]:
arcpy.na.Directions("Vehicle Routing Problem", "TEXT", "D:/git/GIS5572/Lab3/Lab_Report/Directions.txt", "Miles", "REPORT_TIME", "Minutes", "en", "NA Desktop", None)


##exporting directions to a txt.

id,value
0,D:\git\GIS5572\Lab3\Lab_Report\Directions.txt
1,a Layer object


## Discussion

I was able to get a reasonable output in the GUI, but the output from Jupyter shows unreasonable times. I think it is because my time cost from my network dataset could be wrong. The output directions is from a different project and the times look reasonable. 